In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
import librosa
import soundfile as sf
import csv


Mounted at /content/drive/


In [ ]:
import torchaudio

In [ ]:
!pip install speechbrain --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 24.9 MB/s eta 0:00:00


In [ ]:
from speechbrain.inference.speaker import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [15]:
csv_output_path = '/content/drive/MyDrive/embeddings.csv'
directories = [
    '/content/drive/MyDrive/Dysphonia Dutch/Healthy/female_chunked',
    '/content/drive/MyDrive/Dysphonia Dutch/Healthy/male_chunked',
    '/content/drive/MyDrive/Dysphonia Dutch/Patient/female_chunked',
    '/content/drive/MyDrive/Dysphonia Dutch/Patient/male_chunked',

    '/content/drive/MyDrive/Dysphonia English/Healthy/female_chunked',
    '/content/drive/MyDrive/Dysphonia English/Healthy/male_chunked',
    '/content/drive/MyDrive/Dysphonia English/Patient/female_chunked',
    '/content/drive/MyDrive/Dysphonia English/Patient/male_chunked',

    '/content/drive/MyDrive/Dysphonia Hungarian/Healthy/female_chunked',
    '/content/drive/MyDrive/Dysphonia Hungarian/Healthy/male_chunked',
    '/content/drive/MyDrive/Dysphonia Hungarian/Patient/female_chunked',
    '/content/drive/MyDrive/Dysphonia Hungarian/Patient/male_chunked',

    '/content/drive/MyDrive/Dysphonia German/Healthy/Female',
    '/content/drive/MyDrive/Dysphonia German/Healthy/Male',
    '/content/drive/MyDrive/Dysphonia German/Patient/Female',
    '/content/drive/MyDrive/Dysphonia German/Patient/Male',
]


In [ ]:
import torch.nn.functional as F

EXPECTED_SAMPLE_RATE = 16000
EXPECTED_AUDIO_LENGTH = 16000  # 1 second of audio

# Open a CSV file to write the embeddings
with open(csv_output_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(['file_name', 'embedding', 'category', 'gender', 'language'])

    # Loop through each directory and process the files
    for directory in directories:
        for root, dirs, files in os.walk(directory):  # Walk through all subdirectories
            for filename in files:
                if filename.endswith('.wav'):  # Ensure you're processing only .wav files
                    filepath = os.path.join(root, filename)

                    # Load the audio file
                    signal, fs = torchaudio.load(filepath)

                    # Resample if the sample rate is different (GPT)
                    if fs != EXPECTED_SAMPLE_RATE:
                        signal = torchaudio.functional.resample(signal, fs, EXPECTED_SAMPLE_RATE)

                    # Pad or trim the audio to the expected length (GPT)
                    current_length = signal.shape[1]
                    if current_length < EXPECTED_AUDIO_LENGTH:
                        # Pad with zeros
                        pad_length = EXPECTED_AUDIO_LENGTH - current_length
                        signal = F.pad(signal, (0, pad_length))
                    elif current_length > EXPECTED_AUDIO_LENGTH:
                        # Trim to the expected length
                        signal = signal[:, :EXPECTED_AUDIO_LENGTH]

                    # Encode the signal to get embeddings
                    embeddings = classifier.encode_batch(signal)

                    # Squeeze the embeddings to a 512-d vector and convert to a comma-separated string
                    embedding_vector = embeddings.squeeze().tolist()  # Convert to list
                    embedding_string = ','.join(map(str, embedding_vector))  # Convert to comma-separated string

                    # Extract category, gender, and language from the file path
                    # Assuming the directory structure contains the information, e.g., Healthy/male_chunked/English/
                    if 'Healthy' in filepath:
                        category = 'healthy'
                    else:
                        category = 'patient'

                    if 'female' in filepath:
                        gender = 'female'
                    elif 'male' in filepath:
                        gender = 'male'
                    else:
                        gender = 'unknown'

                    if 'English' in filepath:
                        language = 'English'
                    elif 'Dutch' in filepath:
                        language = 'Dutch'
                    elif 'Hungarian' in filepath:
                        language = 'Hungarian'
                    elif 'German' in filepath:
                        language = 'German'
                    else:
                        language = 'unknown'

                    # Write the row (embedding, filename, category, gender, language)
                    csv_writer.writerow([filename, embedding_string, category, gender, language])

print(f"Embeddings have been saved to {csv_output_path}")